In [1]:
%cd /hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning/

import torch
import json
from pathlib import Path

res_dict = {}
predict_dir = "checkpoints/vanilla-llama3-nlp_rw-epoch30-8192-full_finetune/lightning_logs/version_1/predict"
ground_truth_file = "dataset/NLP_rw_new_batch/instruction/test_data_list_1500_8192_ans_gt_256_llama3.pt"

predict_files = list(Path(predict_dir).glob("*.txt"))
for file in predict_files:
    res_dict[int(file.stem)] = {
        "predict": file.read_text()
    }

ground_truths = torch.load(ground_truth_file)
for data_entry in ground_truths:
    if data_entry["id"] in res_dict:
        res_dict[data_entry["id"]]["ground_truth"] = data_entry["ground_truth"]

/hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning


In [2]:
print(len(res_dict))

res_dict = {k: v for k, v in res_dict.items() if "ground_truth" in v}
print(len(res_dict))

500
500


In [4]:
res_dict

{4964: {'predict': 'The Transformer model (@CITE[1]@) and its variants (@CITE[4]@; ; @CITE[6]@) have been widely adopted in NLP and CV tasks. In the Transformer model, the positional encoding is used to encode the position information of tokens. The positional encoding is usually a learnable embedding. In the original Transformer model, the learnable embedding is added to the token embedding to encode the position information. In the BERT model (@CITE[2]@), the absolute position embedding is used to encode the position information of tokens. The absolute position embedding is also used in the DeBERTa model (@CITE[3]@) and the Longformer model (@CITE[5]@). In the Swin Transformer model (@CITE[6]@), the shifted windowing scheme is used to encode the position information of tokens. The shifted windowing scheme is also used in the Swin V2 Transformer model (@CITE[5]@). In our model, we use the shifted windowing scheme to encode the position information of tokens. Our model is different fro

In [3]:
import numpy as np
# calculate the BLEU score with HUGGINGFACE datasets
import evaluate
import os
# os.environ["CUDA_VISIBLE_DEVICES"]= "1"

bleu_score_metric = evaluate.load("bleu")
for id, res in res_dict.items():
    bleu_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
bleu_score = bleu_score_metric.compute()
print("bleu_score: ", bleu_score)

rouge_score_metric = evaluate.load("rouge")
for id, res in res_dict.items():
    rouge_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
rouge_score = rouge_score_metric.compute()
print("rouge_score: ", rouge_score)

bert_score_metric = evaluate.load("bertscore")
for id, res in res_dict.items():
    bert_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
bert_score = bert_score_metric.compute(model_type="distilbert-base-uncased")
print("bert_score: ", np.mean(bert_score['precision']))

/hpc2hdd/home/xzou428/miniconda3/envs/yuh/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bleu_score:  {'bleu': 0.6322909254594729, 'precisions': [0.6549036365967267, 0.6326281529698943, 0.624260761340715, 0.6179820222786117], 'brevity_penalty': 1.0, 'length_ratio': 1.3565167992209057, 'translation_length': 234010, 'reference_length': 172508}
rouge_score:  {'rouge1': 0.8680185422157359, 'rouge2': 0.837546809347565, 'rougeL': 0.8446929613673848, 'rougeLsum': 0.8498592602069218}
bert_score:  0.9612765333652497


In [9]:
import numpy as np
# calculate the BLEU score with HUGGINGFACE datasets
import evaluate

bleu_score_metric = evaluate.load("bleu")
for id, res in res_dict.items():
    bleu_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
bleu_score = bleu_score_metric.compute()
print("bleu_score: ", bleu_score)

rouge_score_metric = evaluate.load("rouge")
for id, res in res_dict.items():
    rouge_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
rouge_score = rouge_score_metric.compute()
print("rouge_score: ", rouge_score)

bert_score_metric = evaluate.load("bertscore")
for id, res in res_dict.items():
    bert_score_metric.add(prediction=res["predict"], reference=res["ground_truth"])
bert_score = bert_score_metric.compute(model_type="distilbert-base-uncased")
print("bert_score: ", np.mean(bert_score['precision']))

bleu_score:  {'bleu': 0.08927876998456319, 'precisions': [0.1245171334234342, 0.08852481468166333, 0.0789748685179792, 0.07298098846279122], 'brevity_penalty': 1.0, 'length_ratio': 4.362359508634715, 'translation_length': 292265, 'reference_length': 66997}
rouge_score:  {'rouge1': 0.35857489742799686, 'rouge2': 0.23754719828266235, 'rougeL': 0.27778690266208955, 'rougeLsum': 0.2972290725568194}


/hpc2hdd/home/xzou428/miniconda3/envs/yuh/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


bert_score:  0.8776230165362358
